In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/140k-real-and-fake-faces/valid.csv
/kaggle/input/140k-real-and-fake-faces/train.csv
/kaggle/input/140k-real-and-fake-faces/test.csv
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/W3DVWMWYOI.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/IQEOWSOAID.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/8JL4TYOH6D.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/69FFH5KFY8.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/ZZAA0631IK.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/BDY8YAWSIS.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/XKHWRIM64I.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/S579BG249Z.jpg
/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/fake/BLIFEE55NI.jpg
/kaggle/input/140k-real-and-fake-fa

In [1]:
import os 
import cv2

In [2]:
os.listdir("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake")

['valid', 'test', 'train']

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Définir le chemin vers le dossier de données
base_dir = "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake"

# Créer des générateurs d'images pour chaque ensemble de données
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalisation des images
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Charger les images et labels pour chaque ensemble de données
train_generator = train_datagen.flow_from_directory(
    directory=f'{base_dir}/train',
    target_size=(224, 224),  # Adapter la taille à celle attendue par votre modèle
    batch_size=32,
    class_mode='binary',     # 'binary' pour deux classes (fake, real)
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    directory=f'{base_dir}/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_directory(
    directory=f'{base_dir}/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.applications import DenseNet121, ResNet50 ,VGG16
from tensorflow.keras.models import Model
import tensorflow as tf

In [8]:
from tensorflow.keras.applications import VGG16
IMAGE_SIZE=224

In [9]:
# Add custom layers on top of VGG16
base_model = tf.keras.applications.VGG16(
    weights='imagenet',  # Utiliser les poids pré-entraînés sur ImageNet
    include_top=False,   # Ne pas inclure la couche de classification supérieure
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)  # Taille de l'entrée adaptée à vos images
)


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers[-3:]:  # Unfreeze the last 4 layers (customize this as needed)
    layer.trainable = True

# Recompile the model after making layers trainable
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # You can reduce the learning rate further if needed
    loss='binary_crossentropy',
    metrics=['accuracy']
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [18]:
# Define callbacks
callbacks = [
    #ModelCheckpoint('vgg16_best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
]

In [19]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=2,
    callbacks=callbacks
)

Epoch 1/2


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1731655083.471031     127 service.cc:145] XLA service 0x7df10c0022b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731655083.471089     127 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1731655103.347873     127 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 984s 307ms/step - accuracy: 0.8222 - loss: 0.3634 - val_accuracy: 0.9694 - val_loss: 0.0857 - learning_rate: 1.0000e-05
Epoch 2/2
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 697s 223ms/step - accuracy: 0.9734 - loss: 0.0749 - val_accuracy: 0.9797 - val_loss: 0.0536 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 2.
